In [56]:
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import joblib

import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_regression,SelectKBest,SelectPercentile,chi2

# declare a two step pipeline, explicitly giving names to both steps.


%matplotlib inline

In [37]:
def trim_set(frame):
    return frame[((frame['TRB'] > 2) | (frame['AST'] > 5)) & (frame['PTS'] > 15)]
    # return frame[((frame['TRB'] > 5) | (frame['AST'] > 5)) & (frame['PTS'] > 15) & (frame['Win%'] > .5)]



In [72]:
df = pd.read_csv('train_set_full.csv', index_col=0)
df.rename(columns={'WS/48':'WS_48'},inplace=True)
print(df.columns)

df = df.drop(columns=['STAR', 'DPOY_Rank', 'DPOY_First', 'DPOY_Pts Won', 'DPOY_Pts Max', 'DPOY_Share',
                      'DPOY', 'MIP_Rank', 'MIP_First', 'MIP_Pts Won', 'MIP_Pts Max',
                      'MIP_Share', 'MIP', 'MVP_First', 'MVP_Pts Won',
                      'MVP_Pts Max', 'ROTY_Rank', 'ROTY_First',
                      'ROTY_Pts Won', 'ROTY_Pts Max', 'ROTY_Share', 'ROTY', 'SMOTY_Rank',
                      'SMOTY_First', 'SMOTY_Pts Won', 'SMOTY_Pts Max', 'SMOTY_Share',
                      'SMOTY'], errors='ignore')

# df = trim_set(df)
df.Year.unique()

Index(['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS_48', 'OBPM', 'DBPM',
       'BPM', 'VORP', 'MVP_Rank', 'MVP_First', 'MVP_Pts Won', 'MVP_Pts Max',
       'MVP_Share'],
      dtype='object')


array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [75]:
df[df.Player.str.contains('Jok')]

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,MVP_Rank,MVP_Share
211,2016,Nikola Jokić,C,20,DEN,80,55,21.7,3.8,7.5,...,4.5,2.2,6.7,0.185,2.6,1.3,3.9,2.6,0.0,0.000
208,2017,Nikola Jokić,C,21,DEN,73,59,27.9,6.8,11.7,...,7.7,2.0,9.7,0.228,5.9,1.4,7.3,4.8,0.0,0.000
233,2018,Nikola Jokić,C,22,DEN,75,73,32.6,6.7,13.5,...,7.8,2.9,10.7,0.211,5.3,1.6,6.9,5.5,0.0,0.000
227,2019,Nikola Jokić,C,23,DEN,80,80,31.3,7.7,15.1,...,7.5,4.3,11.8,0.226,6.2,2.9,9.1,7.0,4.0,0.210
224,2020,Nikola Jokić,C,24,DEN,73,73,32.0,7.7,14.7,...,6.7,3.1,9.8,0.202,5.5,2.0,7.4,5.5,9.0,0.018
222,2021,Nikola Jokić,C,25,DEN,72,72,34.6,10.2,18.0,...,12.2,3.4,15.6,0.301,9.1,3.0,12.1,8.8,1.0,0.961


In [78]:
train_set = df.copy()
test_set = pd.read_csv(f'Next_Year_Predicted_Stats.csv', index_col=0)
test_set = trim_set(test_set)

x_train = df.drop(columns=['MVP_Rank', 'MVP_Share', ])
y_train = train_set['MVP_Share']

test_set.Player.unique()

array(['Bam Adebayo', 'Jarrett Allen', 'Giannis Antetokounmpo',
       'OG Anunoby', 'Deandre Ayton', 'LaMelo Ball', 'Scottie Barnes',
       'RJ Barrett', 'Bradley Beal', 'Bojan Bogdanović', 'Devin Booker',
       'Miles Bridges', 'Malcolm Brogdon', 'Dillon Brooks',
       'Jaylen Brown', 'Jalen Brunson', 'Jimmy Butler'], dtype=object)

In [40]:
# cm = train_set.corr().apply(np.absolute)
# f = plt.figure(figsize=(19, 15))
# plt.matshow(cm.corr(), fignum=f.number)
# plt.xticks(range(df.select_dtypes(['number']).shape[1]), cm.select_dtypes(['number']).columns, fontsize=14, rotation=45)
# plt.yticks(range(df.select_dtypes(['number']).shape[1]), cm.select_dtypes(['number']).columns, fontsize=14)
# cb = plt.colorbar()
# cb.ax.tick_params(labelsize=14, )
# plt.title('Correlation Matrix', fontsize=16)

In [41]:
# feature_list = ['MP', 'PTS', 'TOV', 'AST', 'STL', 'TRB', 'BLK','WS','PER', 'TS%']
# feature_list = ['PTS','PER','TS%','WS','BPM','VORP','USG%']
feature_list = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA',
                'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
                'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
                'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS_48', 'OBPM', 'DBPM',
                'BPM', 'VORP']

x_train = train_set[feature_list]
x_test = test_set[feature_list]

In [64]:

pipe = Pipeline([('feat_selection', SelectPercentile(),),
                 ('scaler', MinMaxScaler()),
                 ('reg', RandomForestRegressor())])

param_grid = [
    {
     'feat_selection':[SelectKBest(),SelectPercentile(percentile=55),],
     # 'feat_selection__percentile':range(45,80,5),
     'scaler':[MinMaxScaler(),StandardScaler()],
     'reg': [RandomForestRegressor(), xgb.XGBRegressor(objective="reg:linear"), SVR(),GradientBoostingRegressor()]

     },
]
ppln_fitted = GridSearchCV(pipe, param_grid, cv=5, verbose=5, n_jobs=-1,return_train_score=True)

ppln_fitted = ppln_fitted.fit(x_train, y_train, )

print(ppln_fitted.best_params_)
print(ppln_fitted.best_score_)
best = ppln_fitted.best_params_


Fitting 5 folds for each of 16 candidates, totalling 80 fits
{'feat_selection': SelectPercentile(percentile=55), 'reg': RandomForestRegressor(), 'scaler': MinMaxScaler()}
0.5978955422502721


In [65]:
y_pred = ppln_fitted.predict(x_test)

test_set['pred'] = y_pred
test_set['pred_scaled'] = MinMaxScaler().fit_transform(y_pred.reshape(-1, 1))
test_set['pred_rank'] = test_set.pred.rank(ascending=False).astype("int")

In [66]:
test_set.sort_values('pred_rank', ascending=True).head(10)[
    ['Player', 'G', 'PTS', 'FG%', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'pred']]

,Player,G,PTS,FG%,AST,TRB,STL,BLK,TOV,pred
11,Giannis Antetokounmpo,92.26,24.904,0.54954,4.413,11.207,1.035,1.309,2.508,0.48309
58,Devin Booker,61.09,26.764,0.44907,4.223,4.948,1.129,0.331,2.560,0.02682
86,Jimmy Butler,48.76,19.773,0.46435,5.488,5.557,1.618,0.558,1.936,0.01028
40,Bradley Beal,32.62,24.358,0.43872,6.281,4.633,0.855,0.309,3.327,0.00531
75,Jaylen Brown,59.49,24.459,0.47368,3.249,6.068,1.220,0.335,2.523,0.00419
24,LaMelo Ball,65.50,20.588,0.43836,7.257,5.908,1.517,0.312,3.117,0.00248
7,Jarrett Allen,44.98,17.673,0.66123,1.490,10.614,0.764,1.269,1.800,0.00027
66,Miles Bridges,72.65,21.242,0.50319,3.204,6.749,0.942,0.724,1.660,0.00018
21,Deandre Ayton,60.08,15.981,0.63165,1.555,10.148,0.636,0.648,1.690,0.00015
2,Bam Adebayo,49.62,16.966,0.55169,3.307,10.119,1.388,0.629,2.333,0.00009


In [57]:

pipe = Pipeline(

    [('feat_selection', SelectPercentile(percentile=60),),('scaler', MinMaxScaler()), ('reg', RandomForestRegressor())])


param_grid = [
    {
     # 'scaler':[MinMaxScaler()],
     'feat_selection__percentile':range(45,60,5),
     'reg': [RandomForestRegressor(), xgb.XGBRegressor(objective="reg:linear"), SVR(),GradientBoostingRegressor()]

     },
]




# param_grid = {
#     'feat_selection':['SelectPercentile':{SelectPercentile(),}
#                       SelectKBest()],
#     'scaler': [MinMaxScaler(), StandardScaler(), ],
#     'reg': [SVR(), GradientBoostingRegressor(), LinearRegression(), ],
#     'feat_selection__percentile': [10,25,50,75,80],
#     'feat_selection__k': [5,10,15,20],
#     {'scaler': [MinMaxScaler()],
#      'feature_range': [(0, 1), (-1, 1)],
#      'knn__n_neighbors': range(1, 10)},
#
#     {'scaler': [StandardScaler()],
#      'knn__n_neighbors': range(1, 10)}
# }

grid = GridSearchCV(pipe, param_grid, cv=5, verbose=5, n_jobs=-1,return_train_score=True)
grid.fit(x_train, y_train)
# print(grid)
print(grid.best_params_)
print(grid.best_score_)
best = grid.best_params_


Fitting 5 folds for each of 12 candidates, totalling 60 fits
{'feat_selection__percentile': 55, 'reg': RandomForestRegressor()}
0.5928802222982398


In [58]:
y_pred = grid.predict(x_test)

test_set['pred'] = y_pred
# test_set['pred_scaled'] = MinMaxScaler().fit_transform(y_pred.reshape(-1, 1))
# test_set['pred_rank'] = test_set.pred.rank(ascending=False).astype("int")


test_set.sort_values('pred_rank', ascending=True).head(10)[
          ['pred_rank','Player', 'G', 'PTS', 'FG%', 'AST', 'TRB', 'STL', 'pred']]

,pred_rank,Player,G,PTS,FG%,AST,TRB,STL,pred
11,1,Giannis Antetokounmpo,92.26,24.904,0.54954,4.413,11.207,1.035,0.53383
58,2,Devin Booker,61.09,26.764,0.44907,4.223,4.948,1.129,0.03313
75,3,Jaylen Brown,59.49,24.459,0.47368,3.249,6.068,1.220,0.01484
86,4,Jimmy Butler,48.76,19.773,0.46435,5.488,5.557,1.618,0.00825
40,5,Bradley Beal,32.62,24.358,0.43872,6.281,4.633,0.855,0.00746
24,6,LaMelo Ball,65.50,20.588,0.43836,7.257,5.908,1.517,0.00188
7,7,Jarrett Allen,44.98,17.673,0.66123,1.490,10.614,0.764,0.00062
21,8,Deandre Ayton,60.08,15.981,0.63165,1.555,10.148,0.636,0.00007
66,9,Miles Bridges,72.65,21.242,0.50319,3.204,6.749,0.942,0.00019
2,10,Bam Adebayo,49.62,16.966,0.55169,3.307,10.119,1.388,0.00016


In [51]:
# ppln_fitted = pipe.fit(x_train, y_train, )

# save
# ppln_fitted = joblib.load("models/RandomForrestRegressor.pkl",)

# load




y_pred = ppln_fitted.predict(x_test)

test_set['pred'] = y_pred
test_set['pred_scaled'] = MinMaxScaler().fit_transform(y_pred.reshape(-1, 1))
test_set['pred_rank'] = test_set.pred.rank(ascending=False).astype("int")


test_set

# test_set.sort_values('pred_rank', ascending=True).head(10)[
#           ['pred_rank','Player', 'G', 'PTS', 'FG%', 'AST', 'TRB', 'STL', 'pred']]

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,...,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,pred,pred_scaled,pred_rank
2,Bam Adebayo,C,25,49.62,41.74,30.975,6.653,12.413,0.55169,0.135,...,2.827,4.642,0.14894,0.011,1.359,1.907,1.290,0.010957,0.220488,12
7,Jarrett Allen,C,24,44.98,37.61,31.616,6.980,10.575,0.66123,0.027,...,2.643,6.570,0.19986,2.724,0.887,4.010,1.844,0.025660,0.268451,9
11,Giannis Antetokounmpo,PF,28,92.26,78.10,24.518,8.101,15.075,0.54954,1.288,...,4.444,14.258,0.29491,5.866,3.746,10.253,7.900,0.249926,1.000000,1
15,OG Anunoby,SF,25,40.24,42.49,34.485,5.828,13.483,0.44031,2.463,...,1.086,2.408,0.08708,-0.768,-0.254,-0.229,0.542,-0.036657,0.065175,16
21,Deandre Ayton,C,24,60.08,56.76,28.446,7.232,11.162,0.63165,0.238,...,3.073,7.305,0.17988,1.577,0.214,2.084,2.252,0.032058,0.289321,7
24,LaMelo Ball,PG,21,65.50,57.00,30.593,7.385,15.885,0.43836,2.796,...,2.233,7.029,0.14381,3.500,-0.162,4.034,3.759,0.079106,0.442788,2
31,Scottie Barnes,PF,21,64.25,61.39,34.634,6.504,13.339,0.50375,0.543,...,2.357,6.644,0.14589,0.859,0.401,1.210,1.598,0.000382,0.185994,14
32,RJ Barrett,SF,22,62.79,56.25,32.635,6.394,15.120,0.40673,2.150,...,1.572,-0.986,0.02169,-1.856,-1.716,-3.007,-1.613,-0.056637,0.000000,17
40,Bradley Beal,SG,29,32.62,20.13,32.152,9.051,19.741,0.43872,1.400,...,0.799,1.191,0.05568,1.957,-1.351,1.145,0.016,0.011589,0.222552,11
54,Bojan Bogdanović,PF,33,60.42,49.39,28.561,5.182,12.102,0.44959,2.550,...,1.833,6.424,0.14194,1.218,-1.308,0.526,1.542,0.010138,0.217818,13


In [48]:
test_set.sort_values('pred_rank', ascending=True).head(10)[
          ['pred_rank','Player', 'PTS', 'TS%', 'AST', 'TRB','USG%','WS_48','BPM','VORP']]

,pred_rank,Player,PTS,TS%,AST,TRB,USG%,WS_48,BPM,VORP
11,1,Giannis Antetokounmpo,24.904,0.63237,4.413,11.207,33.072,0.29491,10.253,7.900
24,2,LaMelo Ball,20.588,0.56522,7.257,5.908,27.749,0.14381,4.034,3.759
66,3,Miles Bridges,21.242,0.59717,3.204,6.749,24.275,0.14597,2.687,2.976
58,4,Devin Booker,26.764,0.55134,4.223,4.948,33.633,0.16244,4.086,3.177
75,5,Jaylen Brown,24.459,0.56380,3.249,6.068,30.899,0.14445,3.077,2.445
86,6,Jimmy Butler,19.773,0.56184,5.488,5.557,25.643,0.21698,5.353,3.644
21,7,Deandre Ayton,15.981,0.65563,1.555,10.148,20.627,0.17988,2.084,2.252
79,8,Jalen Brunson,16.376,0.58142,4.735,3.962,24.072,0.13982,0.874,1.581
7,9,Jarrett Allen,17.673,0.69006,1.490,10.614,20.616,0.19986,4.010,1.844
68,10,Malcolm Brogdon,16.377,0.53405,5.926,4.951,23.513,0.10815,1.459,1.618


In [33]:
# test_set.to_csv(f'Histoic Predictions/{}')
test_set.columns

Index(['Player', 'Pos', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS_48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'pred', 'pred_scaled', 'pred_rank'],
      dtype='object')

In [34]:
test_set

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,...,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,pred,pred_scaled,pred_rank
0,Bam Adebayo,C,26,61.98,56.84,31.933,7.539,14.471,0.52656,0.037,...,3.179,5.216,0.13830,0.895,0.364,1.224,1.690,0.041147,0.267296,36
1,Giannis Antetokounmpo,PF,29,57.64,56.83,31.548,10.580,19.770,0.55233,0.851,...,3.349,7.517,0.21024,4.538,2.475,7.381,4.824,0.201356,1.000000,1
2,OG Anunoby,SF,26,58.28,51.77,32.019,6.191,12.224,0.45874,1.946,...,2.853,5.443,0.10159,-0.317,0.477,-0.103,1.640,-0.014371,0.013386,79
3,Deandre Ayton,C,25,49.44,44.13,29.049,7.206,12.429,0.57084,0.148,...,2.690,3.438,0.13385,0.361,-0.160,0.479,1.350,0.030505,0.218627,44
4,LaMelo Ball,PG,22,50.79,41.17,32.642,7.633,18.845,0.41462,3.333,...,1.938,3.067,0.06096,2.986,-0.395,3.027,1.447,0.055418,0.332564,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Franz Wagner,SF,22,69.49,67.00,31.633,7.321,15.294,0.46731,1.745,...,1.929,5.114,0.09873,0.823,-0.679,0.333,1.036,0.001806,0.087371,67
89,Andrew Wiggins,SF,28,59.25,42.58,28.046,6.309,13.395,0.46612,2.174,...,1.632,3.776,0.08165,-0.886,-0.395,-1.330,0.752,0.006632,0.109444,60
90,Zion Williamson,PF,23,63.38,56.04,32.411,8.733,14.866,0.56214,0.297,...,2.983,8.307,0.17001,4.016,1.005,6.231,3.414,0.069778,0.398235,17
91,Christian Wood,C,28,57.48,32.82,26.963,5.788,11.882,0.48739,1.541,...,1.910,3.542,0.11055,1.197,-0.311,0.983,1.628,0.048677,0.301735,27


In [14]:
df.loc[(df.MVP_Rank == 1) & (df.Year == 1980),'Player'].values[0]

'Kareem Abdul-Jabbar'

In [23]:
df['MVP_Rank'] = df['MVP_Rank'].astype("int")

df.loc[(df.MVP_Rank == 1) & (df.Year == 1997),['Player','MVP']]

KeyError: "['MVP'] not in index"

In [25]:
df = pd.read_csv('train_set_full.csv', index_col=0)

# print(df.columns)

df = df.drop(columns=['STAR', 'DPOY_Rank', 'DPOY_First', 'DPOY_Pts Won', 'DPOY_Pts Max', 'DPOY_Share',
                      'DPOY', 'MIP_Rank', 'MIP_First', 'MIP_Pts Won', 'MIP_Pts Max',
                      'MIP_Share', 'MIP', 'MVP_First', 'MVP_Pts Won',
                      'MVP_Pts Max', 'ROTY_Rank', 'ROTY_First',
                      'ROTY_Pts Won', 'ROTY_Pts Max', 'ROTY_Share', 'ROTY', 'SMOTY_Rank',
                      'SMOTY_First', 'SMOTY_Pts Won', 'SMOTY_Pts Max', 'SMOTY_Share',
                      'SMOTY'], errors='ignore')

# df = trim_set(df)

In [30]:
df[df.Player == 'Karl Malone']

,Year,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,MVP_Rank,MVP_Share
151,1986,Karl Malone,PF,22,UTA,81,76,30.6,6.2,12.5,...,-2.4,4.3,1.9,0.036,-2.4,0.5,-1.9,0.1,0.0,0.000
150,1987,Karl Malone,PF,23,UTA,82,82,34.8,8.9,17.3,...,1.8,5.1,6.9,0.116,0.3,0.0,0.3,1.7,12.0,0.004
144,1988,Karl Malone,PF,24,UTA,82,82,39.0,10.5,20.1,...,4.0,6.1,10.1,0.152,1.7,-0.2,1.5,2.9,8.0,0.045
177,1989,Karl Malone,PF,25,UTA,80,80,39.1,10.1,19.5,...,8.6,6.6,15.2,0.233,4.3,1.2,5.4,5.9,3.0,0.426
185,1990,Karl Malone,PF,26,UTA,82,82,38.1,11.1,19.8,...,10.5,5.4,15.9,0.245,5.9,0.5,6.4,6.6,4.0,0.233
195,1991,Karl Malone,PF,27,UTA,82,82,40.3,10.3,19.6,...,9.9,5.6,15.5,0.225,4.8,0.6,5.4,6.2,5.0,0.148
195,1992,Karl Malone,PF,28,UTA,81,81,37.7,9.9,18.7,...,9.9,5.2,15.1,0.237,5.2,0.6,5.7,5.9,4.0,0.273
203,1993,Karl Malone,PF,29,UTA,82,82,37.8,9.7,17.6,...,10.4,5.0,15.4,0.238,6.0,1.4,7.3,7.3,8.0,0.010
212,1994,Karl Malone,PF,30,UTA,82,82,40.6,9.4,18.9,...,7.4,6.1,13.4,0.193,4.1,1.5,5.6,6.4,7.0,0.017
206,1995,Karl Malone,PF,31,UTA,82,82,38.1,10.1,18.9,...,8.1,5.8,13.8,0.212,4.3,1.3,5.6,6.0,3.0,0.507
